### **Objective:**

The objective of this project is to classify given data as rose and daisy using CNN models.

In [1]:
# Load the following libraries.

import sys
import time
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.utils import get_file

In [2]:
# Load the data.

DATASET_PATH = '../input/flowers-recognition/flowers'

categories = ['daisy', 'rose']

In [3]:
# Load the file names.

fnames = []
for category in categories:
    flower_folder = os.path.join(DATASET_PATH, category)
    file_names = os.listdir(flower_folder)
    full_path = [os.path.join(flower_folder, file_name) for file_name in file_names]
    fnames.append(full_path)

In [4]:
# Determine length of each category.

print('length for each category:', [len(f) for f in fnames])

In [5]:
# Load images.

images = []
for names in fnames:
    one_category_images = [cv2.imread(name) for name in names if (cv2.imread(name)) is not None]
    images.append(one_category_images)

In [6]:
# Determine shape of images.

for i,imgs in enumerate(images):
    shapes = [img.shape for img in imgs]
    widths = [shape[0] for shape in shapes]
    heights = [shape[1] for shape in shapes]
    print('%d,%d is the min shape for %s' % (np.min(widths), np.min(heights), categories[i]))

In [7]:
# Create a function to convert BGR format to RGB format.

def cvtRGB(img):
    return cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)

In [8]:
# Plotting images.

plt.figure(figsize = (15,10))
for i, imgs in enumerate(images):
    plt.subplot(2, 3, i+1)
    idx = np.random.randint(len(imgs))
    plt.imshow(cvtRGB(imgs[idx]))
    plt.grid('off')
    plt.title(categories[i] + ' ' + str(idx))
plt.show()

In [9]:
# Apply resize to all images.

img_width, img_height = 256, 256
resized_images = []
for i,imgs in enumerate(images):
    resized_images.append([cv2.resize(img, (img_width, img_height), interpolation = cv2.INTER_CUBIC) for img in imgs])

In [10]:
# Splitting data to train and test data.

train_images = []
val_images = []
for imgs in resized_images:
    train, test = train_test_split(imgs, train_size = 0.8, test_size = 0.2)
    train_images.append(train)
    val_images.append(test)

In [11]:
# Create labels for all images.

len_train_images = [len(imgs) for imgs in train_images]
print(len_train_images)
print('sum of train images:', np.sum(len_train_images))
train_categories = np.zeros((np.sum(len_train_images)), dtype = 'uint8')
for i in range(5):
    if i is 0:
        train_categories[:len_train_images[i]] = i
    else:
        train_categories[np.sum(len_train_images[:i]):np.sum(len_train_images[:i+1])] = i
        
len_val_images = [len(imgs) for imgs in val_images]
print(len_val_images)
print('sum of val_images:', np.sum(len_val_images))
val_categories = np.zeros((np.sum(len_val_images)), dtype = 'uint8')
for i in range(5):
    if i is 0:
        val_categories[:len_val_images[i]] = i
    else:
        val_categories[np.sum(len_val_images[:i]):np.sum(len_val_images[:i+1])] = i

In [12]:
# Convert images to numpy arrays.

tmp_train_imgs = []
tmp_val_imgs = []
for imgs in train_images:
    tmp_train_imgs += imgs
for imgs in val_images:
    tmp_val_imgs += imgs
train_images = np.array(tmp_train_imgs)
val_images = np.array(tmp_val_imgs)

In [13]:
# Convert class labels to binary class labels.

print('Before converting')
print('train data:', train_images.shape)
print('train labels:', train_categories.shape)

train_data = train_images.astype('float32')
val_data = val_images.astype('float32')
train_labels = np_utils.to_categorical(train_categories, len(categories))
val_labels = np_utils.to_categorical(val_categories, len(categories))
print()
print('After converting')
print('train data:', train_data.shape)
print('train labels:', train_labels.shape)

In [14]:
# Shuffle the data.

seed = 100
np.random.seed(seed)
np.random.shuffle(train_data)
np.random.seed(seed)
np.random.shuffle(train_labels)
np.random.seed(seed)
np.random.shuffle(val_data)
np.random.seed(seed)
np.random.shuffle(val_labels)

In [15]:
# View shape of train_data and test_data.

print('shape of train data:', train_data.shape)
print('shape of train labels:', train_labels.shape)
print('shape of val data:', val_data.shape)
print('shape of val labels:', val_labels.shape)

### **Model Building**

### **1. Create a CNN model**

In [16]:
def create_model_from_scratch():

    model = Sequential()
    model.add(Conv2D(32, (3,3), padding = 'same', input_shape = train_data.shape[1:], activation = 'relu', name = 'conv_1'))
    model.add(Conv2D(32, (3,3), activation = 'relu', name = 'conv_2'))
    model.add(MaxPooling2D(pool_size = (2,2), name = 'maxpool_1'))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding = 'same', activation = 'relu', name = 'conv_3'))
    model.add(Conv2D(64, (3,3), activation = 'relu', name = 'conv_4'))
    model.add(MaxPooling2D(pool_size = (2,2), name = 'maxpool_2'))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3,3), padding = 'same', activation = 'relu', name = 'conv_5'))
    model.add(Conv2D(128, (3,3), activation = 'relu', name = 'conv_6'))
    model.add(MaxPooling2D(pool_size = (2,2), name = 'maxpool_3'))

    model.add(Flatten())
    model.add(Dense(512, activation = 'relu', name = 'dense_1'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation = 'relu', name = 'dense_2'))
    model.add(Dense(len(categories), name = 'output'))
    model.add(Activation('softmax'))

    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc']) 
    
    return model

### **2. Create a VGG19 model**

In [17]:
def create_model_from_VGG19():
    
    model = VGG19(weights = "imagenet", include_top = False, input_shape = (img_width, img_height, 3))
    
    # Freezing the first 5 layers.
    
    for layer in model.layers[:1]:
      layer.trainable = False
      
    # Adding custom Layers. 
    
    x = model.output
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation="relu")(x)
    predictions = Dense(len(categories), activation = "softmax")(x)
    
    # creating the final model.
    
    final_model = Model(inputs = model.input, outputs = predictions)
    
    final_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc']) 
    
    return final_model

### **3. Create ResNet50 model**

In [26]:
def create_model_from_ResNet50():

    model = Sequential()

    model.add(ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet'))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(2048, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(1024, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(len(categories), activation = 'softmax'))

    model.layers[0].trainable = False
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc']) 
    return model

In [19]:
# View model summary.

model_scratch = create_model_from_scratch()

model_scratch.summary()

In [20]:
# View model summary.

model_VGG19 = create_model_from_VGG19()

model_VGG19.summary()

In [27]:
# View model summary.

model_ResNet50 = create_model_from_ResNet50()

model_ResNet50.summary()

### **Data Augmentation**

In [28]:
# Adding rescale, rotation_range, width_shift_range, height_shift_range,
# shear_range, zoom_range, and horizontal flip to ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.4,
    height_shift_range = 0.4,
    shear_range = 0.2,
    zoom_range = 0.3,
    horizontal_flip = True
)


val_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

# Flow training images in batches of 32 using train_datagen generator.

train_generator = train_datagen.flow(
    train_data,
    train_labels,
    batch_size = 32
)

val_generator = val_datagen.flow(
    val_data,
    val_labels,
    batch_size = 32
)

### **Train the model**

In [30]:
batch_size = 32

In [42]:
start = time.time()

model_scratch_info = model_scratch.fit_generator(
    generator = train_generator, 
    steps_per_epoch = len(train_data)/batch_size,   
    epochs = 10, 
    validation_steps = len(val_data)/batch_size, 
    validation_data = val_generator, 
    verbose = 2
)

end = time.time()
duration = end - start
print ('\n model_scratch took %0.2f seconds (%0.1f minutes) to train for %d epochs'%(duration, duration/60, 10) )

In [32]:
start = time.time()

model_VGG19_info = model_VGG19.fit_generator(
    generator = train_generator, 
    steps_per_epoch = len(train_data)/batch_size,   
    epochs = 10, 
    validation_steps = len(val_data)/batch_size, 
    validation_data = val_generator,
    verbose = 2
)

end = time.time()
duration = end - start
print ('\n model_VGG19 took %0.2f seconds (%0.1f minutes) to train for %d epochs'%(duration, duration/60, 10) )

In [33]:
start = time.time()

model_ResNet50_info = model_ResNet50.fit_generator(
    generator = train_generator, 
    steps_per_epoch = len(train_data)/batch_size,   
    epochs = 30, 
    validation_steps = len(val_data)/batch_size, 
    validation_data = val_generator, 
    verbose = 2
)

end = time.time()
duration = end - start
print ('\n model_ResNet50 took %0.2f seconds (%0.1f minutes) to train for %d epochs'%(duration, duration/60, 30))

### **Predict the test data**

In [38]:
def predict_one_image(img, model):
  img = cv2.resize(img, (img_width, img_height), interpolation = cv2.INTER_CUBIC)
  img = np.reshape(img, (1, img_width, img_height, 3))
  img = img/255.
  pred = model.predict(img)
  class_num = np.argmax(pred)
  return class_num, np.max(pred)

In [43]:
# Predict test data using scratch_model.

test_img = cv2.imread('../input/flowers-recognition/flowers/rose/6125332325_b768e08bd9_n.jpg')
pred, probability = predict_one_image(test_img, model_scratch)
print('%s %d%%' % (categories[pred], round(probability, 2) * 100))
_, ax = plt.subplots(1)
plt.imshow(cvtRGB(test_img))
# Turn off tick labels
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.grid('off')
plt.show()

In [40]:
# Predict test data using VGG19 model.

test_img = cv2.imread('../input/flowers-recognition/flowers/rose/6125332325_b768e08bd9_n.jpg')
pred, probability = predict_one_image(test_img, model_VGG19)
print('%s %d%%' % (categories[pred], round(probability, 2) * 100))
_, ax = plt.subplots(1)
plt.imshow(cvtRGB(test_img))
# Turn off tick labels
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.grid('off')
plt.show()

In [39]:
# Predict test data using ResNet50 model.

test_img = cv2.imread('../input/flowers-recognition/flowers/rose/6125332325_b768e08bd9_n.jpg')
pred, probability = predict_one_image(test_img, model_ResNet50)
print('%s %d%%' % (categories[pred], round(probability, 2) * 100))
_, ax = plt.subplots(1)
plt.imshow(cvtRGB(test_img))
# Turn off tick labels
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.grid('off')
plt.show()

### **Conclusion:**

ResNet50 is performing best with an accuracy of 99% when compared to other models.